In [ ]:
import warnings

warnings.filterwarnings("ignore")
from pathlib import Path
import arviz as az

from tbdynamics.calibration.plotting import plot_output_ranges
from tbdynamics.inputs import load_targets, DATA_PATH, DOCS_PATH
import pickle

targets = load_targets()

In [2]:
# OUT_PATH = Path.cwd() / 'runs/r2508'
# loaded_inference_data = az.from_netcdf('inference_data1.nc')
# idata = az.from_netcdf('data/outputs/inference_data.nc')
# params = {
#     "start_population_size": 2000000.0,
#     "seed_time": 1805.0,
#     "seed_num": 1.0,
#     "seed_duration": 1.0,
# }
# quantile_outputs = calculate_scenario_outputs(params, idata)

In [3]:
OUT_PATH = Path(DATA_PATH / 'outputs')

In [4]:
with open(OUT_PATH /'quant_outputs.pkl', 'rb') as f:
    outputs = pickle.load(f)

In [5]:
target_plot = plot_output_ranges(outputs['base_scenario'],targets,['total_population','notification','adults_prevalence_pulmonary'],1,2010,2025)

In [ ]:
target_plot

In [ ]:
# target_plot.write_image(DOCS_PATH / "targets1.png", scale=3)

In [8]:
target_plot_history = plot_output_ranges(outputs['base_scenario'],targets,['total_population','notification','adults_prevalence_pulmonary'],1,1800,2010, history =True)

In [ ]:
target_plot_history

In [9]:
# target_plot_history.write_image(DOCS_PATH / 'targets_history.png', scale=3)

In [10]:
compare_target_plot = plot_output_ranges(outputs['base_scenario'],targets,['incidence','mortality_raw','prevalence_smear_positive', 'percentage_latent'],2,2010,2025)

In [11]:
# compare_target_plot.write_image(DOCS_PATH / "non_targets.png", scale='3')

In [ ]:
compare_target_plot

In [17]:
screening_plot=plot_output_ranges(outputs['base_scenario'],targets,['detection_rate'],1,1981,2025, show_title=False)

In [14]:
# screening_plot.write_image(DOCS_PATH / 'screening_plot.png', scale =3)

In [ ]:
screening_plot